In [ ]:
##### 데이터 다운로드
!pip install datasets   # Package install

from datasets import load_dataset   # Huggingface 데이터셋 패키지 import
data = load_dataset("sepidmnorozy/Korean_sentiment")    # 데이터 다운로드

In [ ]:
##### 코드 상단에서 KoNLPy, Mecab 설치 명령어 실행
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

# Download konlpy
%cd ./drive/MyDrive/Colab\ Notebooks/
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd ./Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

In [ ]:
##### Mecab 동작 확인코드 실행
# from konlpy.tag import Mecab  # KoNLPy를 통해 Mecab 패키지 import
# mecab = Mecab()

# sentence = "실용자연어처리 실습 진행중 이에요. 수업을 시작 할게요"
# print(mecab.morphs(sentence)) # 형태소 출력
# print(mecab.nouns(sentence))  # 명사 출력
# print(mecab.pos(sentence))    # 형태소와 형태소 태그 출력



##### Word embedding
import gensim
import numpy as np

#load word2vec embedding
embedding_model = gensim.models. Word2Vec.load('/content/drive/MyDrive/Colab Notebooks/word2vec/word2vec')
words = embedding_model.wv.index_to_key
vectors = embedding_model.wv.vectors

# 모델에 저장된 단어 10개(0~9)의 단어와 벡터 출력
for i in range(10):
  print(words[i], vectors[i])

#단어 유사도 계산
similarity_dc = embedding_model.wv.similarity('강아지', '고양이')
similarity_ch = embedding_model.wv.similarity('고양이','집')
similarity_dh = embedding_model.wv.similarity('강아지', '집')
print(similarity_dc, similarity_ch, similarity_dh)

1 [-4.27190453e-01  3.20844769e-01 -1.55242786e-01  4.43742782e-01
 -2.26906478e-01  5.67081630e-01  2.28522509e-01  3.13394308e-01
  1.54174492e-01 -8.89793560e-02 -2.29034454e-01  3.01817805e-01
 -5.19342683e-02 -4.95599583e-02  3.93769473e-01  4.39624697e-01
 -3.29991639e-01 -8.23165253e-02 -2.47423649e-01 -2.58893222e-01
  5.07312477e-01  2.27434769e-01  6.47696480e-02  2.86516696e-02
  2.88584501e-01  1.51923314e-01 -3.48429941e-02 -1.88010726e-02
 -2.84610391e-01 -2.70804137e-01 -6.58964634e-01 -1.30652562e-01
 -7.22636878e-02  2.59921283e-01  2.27776423e-01  1.34119481e-01
 -1.30675873e-02  3.45858485e-01 -8.15315545e-02 -1.47723451e-01
  1.71850353e-01  6.36008978e-02  3.75813693e-01 -7.50269666e-02
 -5.19906869e-03  6.31979227e-01  1.91312909e-01  2.43901432e-01
 -1.06249768e-02 -3.34272278e-04  1.28680125e-01  2.87819862e-01
 -3.49104732e-01 -9.04512685e-03 -1.58908665e-01  2.27303416e-01
  2.89688677e-01  3.05232797e-02  5.17697573e-01 -8.80815983e-02
 -1.75788272e-02 -2.440

In [ ]:
##### 데이터 구조 훑어보기
# print("Data type: ", type(data))    # 데이터 타입 확인
# print("Data structure: ", data)     # 데이터 구조 확인
# print("Data keys: ", data.keys())   # 데이터 키 확인

print(data['train'][0])   # 실제 데이터 확인

{'label': 1, 'text': '역시 명작 어렸을때 봤을때도 재밌었고 지금 봐도 몇억배 이상으로 재밌어요'}


In [ ]:
##### 테스트 세트 만들기
train_data = data['train']
dev_data = data['validation']
test_data = data['test']

#print(train_data)
#print(dev_data)
#print(test_data)

# mecab 형태소 분석기 사용
from konlpy.tag import Mecab  # KoNLPy를 통해 Mecab 패키지 import
mecab = Mecab()

# 형태소 단위로 Tokenization 된 텍스트를 {train/dev/test}_data에 저장
train_data = train_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
dev_data = dev_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
test_data = test_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})

print(train_data[0])

{'label': 1, 'text': '역시 명작 어렸 을 때 봤 을 때 도 재밌 었 고 지금 봐도 몇 억 배 이상 으로 재밌 어요'}


In [ ]:
##### 텍스트 데이터 특성
# import matplotlib.pyplot as plt
# plt.hist(data['train']['label'], color='red')
# plt.show()
# plt.hist(data['validation']['label'], color='blue')
# plt.show()
# plt.hist(data['test']['label'], color='green')
# plt.show()

In [ ]:
##### 텍스트와 범주형 특성 다루기 (Bag-of-words 방식으로 벡터화 하는 법 구현)
from sklearn.feature_extraction.text import CountVectorizer       # CountVectorizer: Bag of words 벡터화 구현을 하기 위한 클래스

vectorizer = CountVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+")      # 데이터를 벡터화 해주는 모델
vectorizer.fit(train_data['text'])  # 텍스트 문서 모음을 토큰 수의 행렬로 변환

print(vectorizer.vocabulary_)       # 텍스트 문서에 나타난 어휘의 집합을 출력
print(len(vectorizer.vocabulary_))  # 텍스트 문서에 나타난 어휘 집합의 길이를 출력

train_vectors = vectorizer.transform(train_data['text'])    # 학습 데이터를 숫자로 변환
dev_vectors = vectorizer.transform(dev_data['text'])        # 검증 데이터를 숫자로 변환
test_vectors = vectorizer.transform(test_data['text'])      # 테스트 데이터를 숫자로 변환

# print(train_vectors)      # transform 된 결과 확인


##### 텍스트와 범주형 특성 다루기 (Bag-of-words 방식으로 벡터화 한 것 확인하는 방법 구현)
sample_num = -1                            # 확인하고 싶은 샘플 번호
sample_origin = train_data[sample_num]    # 확인하고 싶은 샘플의 원래 문장
sample_transform = train_vectors[sample_num]    # 확인하고 싶은 샘플의 transform된 결과
sample_inverse_transform = vectorizer.inverse_transform(sample_transform) # 확인하고 싶은 샘플의 transform된 결과를 다시 단어의 조합으로 바꾼 결과
print("Original Input:{}\nTransformed: {}\nInv-transformed: {}".format(sample_origin, sample_transform, sample_inverse_transform))    # 출력

{'역시': 16504, '명작': 9070, '어렸': 15882, '을': 18328, '때': 7257, '봤': 11281, '도': 6341, '재밌': 19803, '었': 16182, '고': 2205, '지금': 21267, '봐도': 11201, '몇': 9086, '억': 16039, '배': 10468, '이상': 18706, '으로': 18227, '어요': 15953, '이런': 18587, '영화': 16757, '는': 5370, '연습': 16572, '용': 17449, '라도': 7550, '만들': 8490, '지': 21245, '말': 8538, '자': 19352, '양심': 15766, '적': 19966, '세대': 13122, '우주': 17599, '를': 8088, '넘나드': 4895, '스카이워커': 13856, '가': 1054, '의': 18409, '비극': 11746, '진짜': 21581, '이': 18458, '20': 114, '여': 16365, '년': 4990, '전': 20000, '에': 16205, '제작': 20494, '되': 6697, '다는': 5556, '게': 1959, '믿기': 9889, '않': 15368, '는다': 5383, '천재': 22260, '감독': 1385, '소소': 13307, '한': 24993, '재미': 19798, '로': 7879, '보': 10935, '꽁': 4101, '트': 23909, '들': 6996, '모아서': 9155, '만든': 8484, '느낌': 5339, '랄까': 7636, 'ᄒᄒ': 24833, '방황': 10466, '하': 24835, '청ᄎ

In [ ]:
##### 훈련세트에서 훈련하고 평가하기
import torch
import torch.nn as nn
import torch.optim as optim

# randomness 제거
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)


# MLP 모델의 아키텍쳐(구조) 정의
class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(MLP, self).__init__()
    # 각 층에 대한 정의
    self.layer1 = nn.Linear(input_size, hidden_size)
    self.layer2 = nn.Linear(hidden_size, int(0.5*hidden_size))
    self.layer3 = nn.Linear(int(0.5*hidden_size), output_size)
    # 활성화 함수
    self.gelu = nn.GELU()
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    # 실제(입력) 데이터: x
    # x를 가지고 순전파를 진행
    out1 = self.layer1(x) #(sum)
    out2 = self.gelu(out1) # hidden_size
    out3 = self.layer2(out2)
    out4 = self.gelu(out3)
    out5 = self.layer3(out4)
    out6 = self.softmax(out5)

    return out6

'''
class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(MLP, self).__init__()
    self.layer1 = nn.Linear(input_size, hidden_size)
    self.layer2 = nn.Linear(hidden_size, int(0.5*hidden_size))
    self.layer3 = nn.Linear(int(0.5*hidden_size), int(2*hidden_size))
    self.layer4 = nn.Linear(int(2*hidden_size), hidden_size)
    self.layer5 = nn.Linear(hidden_size, output_size)
    self.gelu = nn.GELU()
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    out1 = self.layer1(x)
    out2 = self.gelu(out1)
    out3 = self.layer2(out2)
    out4 = self.gelu(out3)
    out5 = self.layer3(out4)
    out6 = self.gelu(out5)
    out7 = self.layer4(out6)
    out8 = self.gelu(out7)
    out9 = self.layer5(out8)
    out10 = self.softmax(out9)

    return out10
'''
# 하이퍼파라미터 셋팅
input_size = len(vectorizer.vocabulary_) # input size
hidden_size = 100 # hidden size
output_size = 2 # output size is 2 (positive/negative)
learning_rate = 0.00001
batch_size = 128
num_epochs = 20

# 모델 초기화
model = MLP(input_size, hidden_size, output_size)
device = torch.device("cuda") # use GPU
model = model.to(device)

# optimizer, loss function 정의
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

# dev_vectors와 dev_data['label']을 텐서 자료형으로 변환
dev_tensors = torch.cuda.FloatTensor(dev_vectors.toarray(), device=device)
dev_labels = torch.tensor(dev_data['label'], dtype=torch.long, device=device)

# 학습 시작 (총 num_epochs 만큼)
for epoch in range(num_epochs):
  # model을 학습모드로 만든다.
  model.train()

  # 학습을 하면서 보고싶은 숫자 (학습이 잘되는지 확인)
  epoch_loss = 0
  best_accuracy = 0

  # train_vectors를 텐서 자료형으로 변환
  train_tensors = torch.cuda.FloatTensor(train_vectors.toarray(), device=device)

  # batch size 단위로 학습 진행
  for i in range(0, len(train_tensors), batch_size):

    # batch 단위 데이터 생성
    batch_data = train_tensors[i:i+batch_size] # batch size 크기의 데이터가 batch_data
    batch_labels = torch.tensor(train_data['label'][i:i+batch_size], device=device)

    # 1. 순전파
    outputs = model(batch_data)
    # 2. 오차 계산
    loss = loss_function(outputs, batch_labels)
    # 3. 역전파
    optimizer.zero_grad()
    # 4. 가중치 업데이트
    loss.backward()
    optimizer.step()

    # epoch loss
    epoch_loss += loss.item()

  # 매 epoch마다 dev 성능 측정
  # 모델을 평가하는 모드로 셋팅
  model.eval()
  with torch.no_grad():
    dev_outputs = model(dev_tensors) # dev 데이터의 순전파
    dev_preds = torch.argmax(dev_outputs, axis=1)
    dev_accuracy = torch.sum(dev_preds == dev_labels).item() / len(dev_labels)

    # save best model on dev data
    if dev_accuracy > best_accuracy:
      best_model = model
      best_accuracy = dev_accuracy

  print(f"Epoch {epoch+1}, Accuracy: {dev_accuracy} , loss: {epoch_loss/len(train_tensors)}")

Epoch 1, Accuracy: 0.5408852213053263 , loss: 0.005426122571031253
Epoch 2, Accuracy: 0.6691672918229558 , loss: 0.005409629429380099
Epoch 3, Accuracy: 0.7449362340585146 , loss: 0.005376305868228276
Epoch 4, Accuracy: 0.7606901725431358 , loss: 0.0053217906223403085
Epoch 5, Accuracy: 0.7756939234808702 , loss: 0.005244214248326089
Epoch 6, Accuracy: 0.7831957989497375 , loss: 0.005146137500802676
Epoch 7, Accuracy: 0.7861965491372843 , loss: 0.005034639288981756
Epoch 8, Accuracy: 0.7891972993248312 , loss: 0.0049181429727209935
Epoch 9, Accuracy: 0.7906976744186046 , loss: 0.0048033254875077145
Epoch 10, Accuracy: 0.7929482370592649 , loss: 0.004694273865885204
Epoch 11, Accuracy: 0.7944486121530383 , loss: 0.004592929065227508
Epoch 12, Accuracy: 0.7996999249812453 , loss: 0.00449982276890013
Epoch 13, Accuracy: 0.8019504876219055 , loss: 0.004414707011646694
Epoch 14, Accuracy: 0.8042010502625656 , loss: 0.004336971521377564
Epoch 15, Accuracy: 0.8079519879969993 , loss: 0.004265

In [ ]:
##### 테스트 세트로 시스템 평가하기
from sklearn.metrics import accuracy_score # Accuracy 측정 함수 import
final_model = best_model # MLP 모델
test_tensors = torch.cuda.FloatTensor(test_vectors.toarray(), device=device)
test_outputs = final_model(test_tensors) # 최종 모델로 test 데이터 예측
pred_results = torch.argmax(test_outputs, axis=1)
accuracy = accuracy_score(test_data['label'], pred_results.tolist()) # 정확도 측정
print("Accuracy: {:.2f}%".format(accuracy*100)) # 정확도 출력

Accuracy: 81.93%
